In [5]:
import os
import sys
import numpy as np
import random

# Root directory of the project
ROOT_DIR = os.path.abspath("../")
sys.path.append(os.path.join(ROOT_DIR, "csci_e89_project/"))

ROOT_IMAGE_DIR = os.path.abspath("images/")

# Path to the dataset (note this is a shared images directory)
dataset_path = os.path.join(ROOT_IMAGE_DIR, "imgnet_n02114100")

models_dir = os.path.join(ROOT_DIR,  "csci_e89_project/models/")

print("base dataset dir:", dataset_path)
print("base modesl dir:", models_dir)

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import model as modellib, utils


base dataset dir: /home/saad/notebook/csci_e89/project/saad_mrcnn/csci_e89_project/images/imgnet_n02114100
base modesl dir: /home/saad/notebook/csci_e89/project/saad_mrcnn/csci_e89_project/models/


In [6]:
import det

class InferenceConfig(det.DetConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 2 # For BG + Others
    
inf_config = InferenceConfig()
inf_model = modellib.MaskRCNN(mode="inference", 
                              config=inf_config,
                              model_dir=models_dir)

# Use known model
weights_path = os.path.join(models_dir, "wolf20180508T1124_mask_rcnn_wolf_0075.h5")

print("Using latest weights: ", weights_path)

inf_model.load_weights(weights_path, by_name=True)


Using latest weights:  /home/saad/notebook/csci_e89/project/saad_mrcnn/csci_e89_project/models/wolf20180508T1124_mask_rcnn_wolf_0075.h5


In [7]:
_, dataset_val = det.create_datasets(dataset_path+'/train', inf_config.CLASS_NAMES)
dataset_val.prepare()


Adding class   1:wolf
Adding class   1:wolf


In [8]:
# Compute VOC-Style mAP @ IoU=0.5

# Since we have a small dataset, run on all
n_image_ids = len(dataset_val.image_ids)
image_ids = np.random.choice(dataset_val.image_ids, n_image_ids)
APs = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inf_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inf_config), 0)
    # Run object detection
    results = inf_model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)
    
print("mAP: {:.4f}".format(np.mean(APs)))

mAP: 0.9048
